<a href="https://colab.research.google.com/github/yasis36/Wheat-leaf-disease-classification-using-CNN-/blob/main/WLD_Classification_using_CNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow numpy matplotlib


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import numpy as np
import os
# Define directories
base_dir = os.path.join(os.path.expanduser('~') ,'/content/drive/MyDrive/Wheat leaf Disease classification')
train_dir = os.path.join(base_dir, '/content/drive/MyDrive/Wheat leaf Disease classification/train')
validation_dir = os.path.join(base_dir, '/content/drive/MyDrive/Wheat leaf Disease classification/validation')


if not os.path.exists(train_dir):
    print(f"Train directory '{train_dir}' not found.")
    exit(1)


# Ensure validation_dir exists
if not os.path.exists(validation_dir):
    print(f"Validation directory '{validation_dir}' not found.")
    exit(1)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
    shuffle=False
)
model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', kernel_regularizer=l2(0.001), input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes, activation='softmax', kernel_regularizer=l2(0.001)))

model.summary()

# Model compile

model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)
# Save model
model.save('/content/drive/MyDrive/Wheat leaf Disease classification/wheat_leaf_disease_classification_model.h5')

Found 13820 images belonging to 4 classes.
Found 1412 images belonging to 4 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 52, 52, 128)       73856     
                                   

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Get the true labels and predictions
Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = validation_generator.classes

# Generate a classification report
print(classification_report(y_true, y_pred, target_names=validation_generator.class_indices.keys()))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

Epoch 1/30
150/150 [==============================] - 958s 6s/step - loss: 1.0527 - accuracy: 0.5404 - val_loss: 1.8660 - val_accuracy: 0.3214
Epoch 2/30
150/150 [==============================] - 224s 1s/step - loss: 0.6197 - accuracy: 0.7729 - val_loss: 2.1301 - val_accuracy: 0.3527
Epoch 3/30
150/150 [==============================] - 230s 2s/step - loss: 0.4691 - accuracy: 0.8283 - val_loss: 2.6483 - val_accuracy: 0.3571
Epoch 4/30
150/150 [==============================] - 224s 1s/step - loss: 0.3973 - accuracy: 0.8583 - val_loss: 3.1273 - val_accuracy: 0.3438
Epoch 5/30
150/150 [==============================] - 222s 1s/step - loss: 0.3546 - accuracy: 0.8729 - val_loss: 3.0125 - val_accuracy: 0.3259
Epoch 6/30
150/150 [==============================] - 231s 2s/step - loss: 0.2988 - accuracy: 0.8963 - val_loss: 3.9626 - val_accuracy: 0.3750
Epoch 7/30
150/150 [==============================] - 222s 1s/step - loss: 0.2625 - accuracy: 0.9125 - val_loss: 3.5968 - val_accuracy: 0.3482

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluate the model
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation accuracy: {val_acc:.2f}")

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Generate predictions
validation_generator.reset()
predictions = model.predict(validation_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Print confusion matrix
print(conf_matrix)


NameError: name 'validation_generator' is not defined

In [ ]:
pip install scikit-learn seaborn


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Generate predictions
validation_generator.reset()
predictions = model.predict(validation_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)


NameError: name 'validation_generator' is not defined

In [ ]:
# Classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)


NameError: name 'true_classes' is not defined